<a href="https://colab.research.google.com/github/LuViBeBe93/tesis/blob/main/GPT2_F5_op.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers pandas torch bert-score nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00


In [2]:
# Importación de bibliotecas
from google.colab import files
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModelWithLMHead, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score
import math

In [3]:
# Subir el archivo de datos
uploaded = files.upload()
data = pd.read_excel('BASE_30122024.xlsx')
data = pd.DataFrame(data)

Saving BASE_30122024.xlsx to BASE_30122024.xlsx


In [4]:
# Configuración para usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Convertir datos numéricos a texto
def datos_a_texto(row):
    texto_datos = (
        f"Año: {row.get('Año', 'NaN')}, Mes: {row.get('Mes', 'NaN')}, "
        f"Trimestre: {row.get('Trimestre', 'NaN')}, Frecuencia: {row.get('Frecuencia', 'NaN')}, "
        f"Año_comparación: {row.get('Año_comparación', 'NaN')}, País: {row.get('País', 'NaN')}, "
        f"Tendencia: {row.get('Tendencia', 'NaN')}, Sector: {row.get('Sector', 'NaN')}, "
        f"Indicador: {row.get('Indicador', 'NaN')}, Valor_actual: {row.get('Valor_actual', 'NaN')}, "
        f"Variación: {row.get('Variación', 'NaN')}, Valor_comparación: {row.get('Valor_comparación', 'NaN')}, "
        f"Entidad: {row.get('Entidad', 'NaN')}"
    )
    return texto_datos

# Crear una columna con los datos numéricos convertidos a texto
data["entrada_texto"] = data.apply(datos_a_texto, axis=1)
data["texto_final"] = data["entrada_texto"] + " </s> Noticia: " + data["Noticia"]


In [6]:
# Dataset personalizado
class NoticiasDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        texto = self.data.iloc[idx]["texto_final"]
        encoding = self.tokenizer(
            texto,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        input_ids = encoding["input_ids"].flatten()
        attention_mask = encoding["attention_mask"].flatten()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids,  # En GPT, las etiquetas son las mismas que la entrada
        }

In [7]:
# Dividir datos en 80% entrenamiento y 20% evaluación
train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)


In [8]:
# Inicializar el tokenizador y modelo GPT-2
tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")
model = AutoModelWithLMHead.from_pretrained("datificate/gpt2-small-spanish")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1838: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [9]:
# Ajustar el tokenizador para manejar padding
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [10]:
# Crear datasets y loaders
train_dataset = NoticiasDataset(train_data, tokenizer)
eval_dataset = NoticiasDataset(eval_data, tokenizer)

In [11]:
# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir="./gpt2_noticias_model",
    num_train_epochs=7, #Este parámetro sale de la optimización bayesiana con Optuna
    per_device_train_batch_size=4, #Este parámetro sale de la optimización bayesiana con Optuna
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
)

# Crear el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Entrenar el modelo
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.326800,0.269236
2,0.258000,0.230041
3,0.230900,0.216880
4,0.210200,0.203514
5,0.203400,0.199466
6,0.190800,0.195016
7,0.185500,0.192351


TrainOutput(global_step=4466, training_loss=0.2635181626022209, metrics={'train_runtime': 963.3266, 'train_samples_per_second': 18.53, 'train_steps_per_second': 4.636, 'total_flos': 1166015692800000.0, 'train_loss': 0.2635181626022209, 'epoch': 7.0})

In [12]:
def generar_noticia(model, tokenizer, datos):
    entrada_texto = datos_a_texto(datos)
    inputs = tokenizer(
        entrada_texto,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(model.device)

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Configurar pad_token_id solo si no está configurado
    if model.config.pad_token_id is None:
        model.config.pad_token_id = tokenizer.eos_token_id  # Asigna eos_token_id si no existe un pad_token

    # Generar texto
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,  # Pasar el `attention_mask`
        max_length=128,
        num_beams=3, #Este parámetro sale de la optimización bayesiana con Optuna
        no_repeat_ngram_size=3, #Este parámetro sale de la optimización bayesiana con Optuna
        early_stopping=True,
        temperature=0.7,  #Este parámetro sale de la optimización por grilla
        top_p=0.9,  #Este parámetro sale de la optimización por grilla
        do_sample=True,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [13]:
# Evaluación con métricas BLEU, BERTScore y Perplejidad
def calcular_bleu(predicciones, referencias):
    smoothing_function = SmoothingFunction().method4
    scores = []
    for pred, ref in zip(predicciones, referencias):
        ref_tokens = [ref.split()]
        pred_tokens = pred.split()
        score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothing_function)
        scores.append(score)
    return scores

def calcular_bertscore(predicciones, referencias, lang="es"):
    P, R, F1 = score(predicciones, referencias, lang=lang)
    return P.tolist(), R.tolist(), F1.tolist()

def calcular_perplejidad(model, tokenizer, textos):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for texto in textos:
            inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)
            input_ids = inputs["input_ids"].to(model.device)
            attention_mask = inputs["attention_mask"].to(model.device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss.item()
            total_loss += loss
    avg_loss = total_loss / len(textos)
    return math.exp(avg_loss)

# Generar predicciones en el conjunto de evaluación
predicciones_generadas = [
    generar_noticia(model, tokenizer, datos) for _, datos in eval_data.iterrows()
]
referencias_reales = eval_data["Noticia"].tolist()


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [14]:
# Calcular métricas
bleu_scores = calcular_bleu(predicciones_generadas, referencias_reales)
precision, recall, f1 = calcular_bertscore(predicciones_generadas, referencias_reales, lang="es")
perplejidad = calcular_perplejidad(model, tokenizer, referencias_reales)

# Mostrar resultados
print("\nResultados de Evaluación:")
print("BLEU Promedio:", sum(bleu_scores) / len(bleu_scores))
print("BERTScore Precisión Promedio:", sum(precision) / len(precision))
print("BERTScore Recall Promedio:", sum(recall) / len(recall))
print("BERTScore F1 Promedio:", sum(f1) / len(f1))
print("Perplejidad:", perplejidad)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Resultados de Evaluación:
BLEU Promedio: 0.15239697560815846
BERTScore Precisión Promedio: 0.6006049802497636
BERTScore Recall Promedio: 0.8382090233896967
BERTScore F1 Promedio: 0.6991741194807249
Perplejidad: 12.751651932857213
